In [2]:
print "IMPORTING LIBRARIES..."
import pandas as pd
import numpy as np
import statsmodels.api as sm
import re
import requests
from requests.auth import HTTPBasicAuth


#DOWLOADING FILE FROM DROPBOX FIRST TIME
import urllib2
import os.path
import time
import random
while not os.path.exists('dev.csv') or not os.path.exists('oot0.csv'):
    time.sleep (3*random.random()); #Sleeping less than 3 seconds before going to Dropbox - avoid too many students at once.
    if not os.path.exists('dev.csv'):
        print "DOWLOADING FILE dev.csv FROM DROPBOX BECAUSE LOCAL FILE DOES NOT EXIST!"
        csvfile = urllib2.urlopen("https://dl.dropboxusercontent.com/u/28535341/dev.csv")
        output = open('dev.csv','wb')
        output.write(csvfile.read())
        output.close()
    if not os.path.exists('oot0.csv'):
        print "DOWLOADING FILE oot0.csv FROM DROPBOX BECAUSE LOCAL FILE DOES NOT EXIST!"
        csvfile = urllib2.urlopen("https://dl.dropboxusercontent.com/u/28535341/oot0.csv")
        output = open('oot0.csv','wb')
        output.write(csvfile.read())
        output.close()  
#DOWLOADING FILE FROM DROPBOX FIRST TIME

    
print "LOADING DATASETS..."
df = pd.read_csv("dev.csv") #DEV-SAMPLE
dfo = pd.read_csv("oot0.csv")#OUT-OF-TIME SAMPLE

print "IDENTIFYING TYPES..."
in_model = []
list_ib = set()  #input binary
list_icn = set() #input categorical nominal
list_ico = set() #input categorical ordinal
list_if = set()  #input numerical continuos (input float)
list_inputs = set()
output_var = 'ob_target' # result used later
for var_name in df.columns:
    if re.search('^i',var_name):
        list_inputs.add(var_name)
        print var_name,"is input"   
    if re.search('^ib_',var_name):
        list_ib.add(var_name)
        print var_name,"is input binary"
    elif re.search('^icn_',var_name):
        list_icn.add(var_name)
        print var_name,"is input categorical nominal"
    elif re.search('^ico_',var_name):
        list_ico.add(var_name)
        print var_name,"is input categorical ordinal"
    elif re.search('^if_',var_name):
        list_if.add(var_name)
        print var_name,"is input numerical continuos (input float)"
    elif re.search('^ob_',var_name):
        output_var = var_name
    else:
        print "ERROR: unable to identify the type of:", var_name

IMPORTING LIBRARIES...
LOADING DATASETS...
IDENTIFYING TYPES...
id is input
ERROR: unable to identify the type of: id
ib_var_1 is input
ib_var_1 is input binary
ib_var_2 is input
ib_var_2 is input binary
ib_var_3 is input
ib_var_3 is input binary
ib_var_4 is input
ib_var_4 is input binary
ib_var_5 is input
ib_var_5 is input binary
ib_var_6 is input
ib_var_6 is input binary
ib_var_7 is input
ib_var_7 is input binary
ib_var_8 is input
ib_var_8 is input binary
ib_var_9 is input
ib_var_9 is input binary
ib_var_10 is input
ib_var_10 is input binary
ib_var_11 is input
ib_var_11 is input binary
ib_var_12 is input
ib_var_12 is input binary
ib_var_13 is input
ib_var_13 is input binary
ib_var_14 is input
ib_var_14 is input binary
ib_var_15 is input
ib_var_15 is input binary
ib_var_16 is input
ib_var_16 is input binary
ib_var_17 is input
ib_var_17 is input binary
ib_var_18 is input
ib_var_18 is input binary
ib_var_19 is input
ib_var_19 is input binary
ib_var_20 is input
ib_var_20 is input binary


In [3]:
print "STEP 1: DOING MY TRANSFORMATIONS..."

def psi(bench, comp, group):
    ben_len=len(bench);
    comp_len=len(comp);
    bench.sort();
    comp.sort();
    psi_cut=[];
    n=int(math.floor(ben_len/group));
    for i in range(1,group):
        lowercut=bench[(i-1)*n+1];
        if i!=group:
            uppercut=bench[(i*n)];
            ben_cnt=n;
        else:
            uppercut=bench[-1];
            ben_cnt=ben_len-group*(n-1)
    comp_cnt = len([i for i in comp if i > lowercut and i<=uppercut]);
    ben_pct=(ben_cnt+0.0)/ben_len;
    comp_pct=(comp_cnt+0.0)/comp_len;
    if comp_pct > 0.0:
        psi_cut.append((ben_pct-comp_pct)*math.log(ben_pct/(comp_pct)));
    else:
        psi_cut.append(0);
    psi=sum(psi_cut);
    return psi;
    
    
#Será necesario transformar las variables categoricas ordinales, en variables 
print " ------------ PSI for DEV dataset-------------"
import re
import math
list_inputs = set()
var_to_check = set()
for var_name in df.columns:
    if re.search('^i',var_name):
        list_inputs.add(var_name)
for var_name in list_inputs:
    psi_value=psi(bench=list(df[var_name]),comp=list(df[var_name]),group=max(2,min((len(set(df[var_name]))),10)));
    if psi_value > 0.1:
        print "psi for ", var_name, " = ", psi_value
        var_to_check.add(var_name)

print " ------------ PSI for OOT dataset-------------"
list_inputs = set()
var_to_check = set()
for var_name in dfo.columns:
    if re.search('^i',var_name):
        list_inputs.add(var_name)
for var_name in list_inputs:
    psi_value=psi(bench=list(dfo[var_name]),comp=list(dfo[var_name]),group=max(2,min((len(set(dfo[var_name]))),10)));
    if psi_value > 0.1:
        print "psi for ", var_name, " = ", psi_value
        var_to_check.add(var_name)

#dfo1 = dfo.fillna(dfo.mean())
#dfo["if_var_78"] = dfo["if_var_78"].fillna(dfo["if_var_78"].mean())
#dfo['ico_var_33'] = dfo['ico_var_33'].fillna(dfo['ico_var_33'].median())
#print list_inputs

STEP 1: DOING MY TRANSFORMATIONS...
 ------------ PSI for DEV dataset-------------
psi for  ib_var_20  =  0.17451251714
psi for  ib_var_19  =  0.275016288497
psi for  ib_var_15  =  0.230828737949
psi for  ib_var_16  =  0.208713388894
psi for  ib_var_17  =  0.267449779291
 ------------ PSI for OOT dataset-------------


In [4]:
#definimos una funion para ver que valor incoporamos a cada variable, si el minimo, el máximo o la media
from sklearn.metrics import roc_auc_score
def generate_model(var, model):
    model = RandomForestClassifier(n_estimators=150, criterion='gini')
    for i in range(2):
        dftemp = dfo
        operation=''
        if i==0:
            dftemp[var] = dftemp[var].fillna(dfo[var].median())
            oper = 'median'
        if i==1:
            dftemp[var] = dftemp[var].fillna(dfo[var].min())
            oper = 'min'
        if i==2:
            dftemp[var] = dftemp[var].fillna(dfo[var].max())
            oper = 'max'
        
        X = df[var]
        Xo = dftemp[var]
        y = df[output_var]
        gini_score = 0
        try:
            print "TRYING: variable",var," WITH OPERATION:",result, Xo
            rfcl = model.fit(X, y)
            yo  = result.predict(Xo)
            dftemp['pred'] = yo
            dftemp_tosend = dftemp[list(['id','pred'])]
            i=1
            filename = "group_Z_sub"+str(i)+".csv"
            dfo_tosend.to_csv(filename, sep=',')
            url = 'http://mgadi.pythonanywhere.com/api/v1.0/uploadpredictions'
            files = {'file': (filename, open(filename, 'rb'))}
            rsub = requests.post(url, files=files, auth=HTTPBasicAuth('amarino', 'amarino'))
            resp_str = str(rsub.text)
            gini = float(resp_str.split(';')[1].split('=')[1].strip())
            if gini>gini_score:
                gini_score=gini
                operation = oper
            print "RESULT GET: variable",var," WITH OPERATION:",result, resp_str
            time.sleep(3)

        except np.linalg.linalg.LinAlgError as err:
            if 'Singular matrix' in err.message:
                print "MODEL-INVALID (Singular Matrix)"
            else:
                raise
    return operation

# list_inputs = list(set(list_inputs))
# X = df[list_inputs]
# model = RandomForestClassifier(n_estimators=150, criterion='gini')
# rfcl = model.fit(X, y)
##Eliminamos valores nulos.
# for column in list_inputs:
#     if dfo[column].isnull().any():
#         #operation = generate_model(column, rfcl)
#         if operation == 'median':
#             dfo[column] = dfo[column].fillna(dfo[column].median())
#         if operation == 'max':
#             dfo[column] = dfo[column].fillna(dfo[column].max())
#         if operation == 'min':
#             dfo[column] = dfo[column].fillna(dfo[column].min())
for column in list_inputs:
    if dfo[column].isnull().any():
            dfo[column] = dfo[column].fillna(dfo[column].median())

In [5]:
#Comparamos valores fuera de rango
for column in list_inputs:
    dfo[dfo[column]>max(df[column])][column] = max(df[column])
    dfo[dfo[column]<min(df[column])][column] = min(df[column])

C:\Desarrollo\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Desarrollo\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
dfo[dfo['if_var_65']<min(df['if_var_65'])]['if_var_65']
min(df['if_var_65'])

21

In [7]:
dfo.isnull().any().any()

False

In [8]:
print "STEP 2: SELECTING CHARACTERISTICS TO ENTER INTO THE MODEL..."
#in_model = list_inputs #['ib_var_1','icn_var_22','ico_var_25','if_var_65']
#in_model = ['ib_var_1','icn_var_22','ico_var_25','if_var_65']
####-------------------------------------------#####
from sklearn.ensemble import RandomForestClassifier

list_inputs = list(set(list_inputs))
X = df[list_inputs]
y = df[output_var]

print "-------STEP 2.1: FEATURE IMPORTANCE WITH RANDOM FOREST---------"
# Build a forest and compute the feature importances
forest = RandomForestClassifier(n_estimators=250,
                              random_state=0)
forest.fit(X, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
# Print the feature ranking
in_model = set()
print("Feature ranking:")
for f in range(40):
    print("%d. feature %s (%f) " % (f + 1, list_inputs[indices[f]], importances[indices[f]] ))
    in_model.add(list_inputs[indices[f]])

in_model = list(in_model) 
#print in_model
####-------------------------------------------#####

STEP 2: SELECTING CHARACTERISTICS TO ENTER INTO THE MODEL...
-------STEP 2.1: FEATURE IMPORTANCE WITH RANDOM FOREST---------
Feature ranking:
1. feature if_var_68 (0.041134) 
2. feature if_var_78 (0.038343) 
3. feature id (0.037014) 
4. feature if_var_65 (0.036702) 
5. feature if_var_79 (0.035192) 
6. feature if_var_69 (0.030505) 
7. feature ico_var_34 (0.028048) 
8. feature if_var_76 (0.027751) 
9. feature ico_var_35 (0.027401) 
10. feature if_var_75 (0.026985) 
11. feature if_var_77 (0.026701) 
12. feature if_var_67 (0.026019) 
13. feature ico_var_36 (0.024158) 
14. feature if_var_72 (0.022102) 
15. feature if_var_80 (0.022067) 
16. feature if_var_73 (0.021330) 
17. feature if_var_71 (0.019819) 
18. feature ico_var_33 (0.018523) 
19. feature if_var_81 (0.017479) 
20. feature ico_var_42 (0.014560) 
21. feature if_var_70 (0.013803) 
22. feature ico_var_41 (0.013643) 
23. feature icn_var_24 (0.013612) 
24. feature ico_var_52 (0.013586) 
25. feature ico_var_56 (0.013072) 
26. feature ico

In [9]:
print "STEP 3: DEVELOPING THE MODEL..."
X = df[list(in_model)]
y = df[output_var]
Xo = dfo[list(set(in_model))]

#model = sm.Logit(y,X)

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

try:
#     model = sm.GLM(y,X, family = sm.families.Binomial())
#     result = model.fit()
#     print result.summary()
#     y_pred = result.predict(X)
#     yo_pred = result.predict(Xo)

#     model = tree.DecisionTreeClassifier()
#     result = model.fit(X, y)
#     y_pred  = result.predict_proba(X)
#     y_pred = y_pred[:,1]
#     yo_pred = result.predict_proba(Xo)
#     yo_pred = yo_pred[:,1]

    model = RandomForestClassifier(n_estimators=150, criterion='gini')
    result = model.fit(X, y)
    #print result.summary()
    y_pred  = result.predict_proba(X)
    y_pred = y_pred[:,1]
    yo_pred = result.predict_proba(Xo)
    yo_pred = yo_pred[:,1]
    
except np.linalg.linalg.LinAlgError as err:
    if 'Singular matrix' in err.message:
        print "MODEL-INVALID (Singular Matrix)"
    else:
        raise
#Generamos una funcion para hacer el stepwise        
def generate_model(in_model):
    X = df[list(in_model)]
    y = df[output_var]
    Xo = dfo[list(set(in_model))]  
    try:
        model = RandomForestClassifier(n_estimators=150, criterion='gini')
        result = model.fit(X, y)
        #print result.summary()
        y_pred  = result.predict_proba(X)
        y_pred = y_pred[:,1]
        yo_pred = result.predict_proba(Xo)
        yo_pred = yo_pred[:,1]

    except np.linalg.linalg.LinAlgError as err:
        if 'Singular matrix' in err.message:
            print "MODEL-INVALID (Singular Matrix)"
        else:
            raise
    return yo_pred

STEP 3: DEVELOPING THE MODEL...


In [10]:
print "STEP 4: ASSESSING THE MODEL..."
# CALCULATING GINI PERFORMANCE ON DEVELOPMENT SAMPLE
from sklearn.metrics import roc_auc_score
gini_score = 2*roc_auc_score(df[output_var], y_pred)-1
print "GINI DEVELOPMENT=", gini_score


STEP 4: ASSESSING THE MODEL...
GINI DEVELOPMENT= 1.0


In [11]:
print "STEP 5: SUBMITTING THE RESULTS..."
dfo['pred'] = yo_pred
dfo_tosend = dfo[list(['id','pred'])]

i=1
filename = "group_Z_sub"+str(i)+".csv"
dfo_tosend.to_csv(filename, sep=',')

url = 'http://mgadi.pythonanywhere.com/api/v1.0/uploadpredictions'

files = {'file': (filename, open(filename, 'rb'))}
rsub = requests.post(url, files=files, auth=HTTPBasicAuth('amarino', 'amarino'))
resp_str = str(rsub.text)
print "RESULT SUBMISSION: ", resp_str

STEP 5: SUBMITTING THE RESULTS...
RESULT SUBMISSION:  KS2 = 0.269547791194; GINI = 0.350064257845


In [12]:
float(resp_str.split(';')[1].split('=')[1].strip())

0.350064257845